<a href="https://colab.research.google.com/github/iptimoshenko/ai-powered-education-app/blob/main/Convert_audio_to_text.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Step 2: Set up Google Cloud Storage and Google Speech-to-Text API
# You will need to set up Google Cloud credentials and install necessary libraries
# Refer to Google Cloud documentation for detailed setup instructions

# Assuming you have the Google Cloud SDK and necessary libraries installed
from google.cloud import storage
from google.cloud import speech_v1p1beta1 as speech
import os
import json

# Set your Google Cloud credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = 'path_to_your_google_cloud_credentials.json'

# Initialize Google Cloud Storage and Speech-to-Text clients
storage_client = storage.Client()
speech_client = speech.SpeechClient()

# Function to upload file to Google Cloud Storage
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)

# Function to transcribe audio file
def transcribe_audio(gcs_uri):
    audio = speech.RecognitionAudio(uri=gcs_uri)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=16000,
        language_code="en-US",
        enable_automatic_punctuation=True
    )

    operation = speech_client.long_running_recognize(config=config, audio=audio)
    response = operation.result(timeout=90)

    # Extracting text from response
    transcript = ""
    for result in response.results:
        transcript += result.alternatives[0].transcript + "\n"

    return transcript

# Step 3: Process files incrementally
# Assuming you have a list of file paths in your Google Drive folder
audio_files = ['path_to_your_audio_file1.wav', 'path_to_your_audio_file2.wav', ...]

# Process 59 minutes per day
time_processed_today = 0
max_daily_processing_time = 59 * 60  # 59 minutes in seconds

for file_path in audio_files:
    # Check if time limit is reached
    if time_processed_today >= max_daily_processing_time:
        break

    # Upload the file to Google Cloud Storage
    # You need to define the bucket name and create a unique blob name for each file
    bucket_name = 'your_bucket_name'
    blob_name = os.path.basename(file_path)
    upload_blob(bucket_name, file_path, blob_name)

    # Transcribe the audio file
    gcs_uri = f'gs://{bucket_name}/{blob_name}'
    text_output = transcribe_audio(gcs_uri)

    # Save the output in JSON format in another folder on Google Drive
    output_path = '/content/drive/My Drive/your_output_folder/' + blob_name.replace('.wav', '.json')
    with open(output_path, 'w') as json_file:
        json.dump({'transcript': text_output}, json_file)

    # Update time processed today
    # You will need to calculate the duration of the current audio file
    # and add it to time_processed_today

# Step 4: Implement File Splitting and Naming Convention
# If a file's duration exceeds the remaining processing time for the day,
# split the file, process the allowable part, and continue with the rest the next day.
# Use a naming convention like 'filename_1', 'filename_2', etc., for split files.
